# Utiliser des décorateurs pour améliorer notre code

Dans ce guide, nous allons explorer les décorateurs et voir comment ils sont utiles dans de nombreuses situations où nous souhaitons améliorer notre conception. Nous commencerons par explorer ce que sont les décorateurs, comment ils fonctionnent et comment ils sont mis en œuvr.

Forts de ces connaissances, nous revisiterons ensuite les concepts que nous avons appris dans les guide précédents concernant les bonnes pratiques générales pour la conception de logiciels et verrons comment les décorateurs peuvent nous aider à respecter chaque principe.

Les objectifs de ce guide sont les suivants :

* Comprendre comment fonctionnent les décorateurs en Python

*  Apprendre à implémenter des décorateurs qui s'appliquent aux fonctions et aux classes

* Implémenter efficacement les décorateurs, en évitant les erreurs d'implémentation courantes

* Analyser comment éviter la duplication de code (le Principe DRY) avec les décorateurs 

* Pour étudier comment les décorateurs contribuent à la séparation des préoccupations

* Pour analyser des exemples de bons décorateurs

* Pour revoir les situations, idiomes ou modèles courants pour savoir quand les décorateurs sont le bon choix

## Que sont les décorateurs en Python ?

Les décorateurs ont été introduits en Python il y a longtemps, dans PEP-318, en tant que mécanisme pour simplifier la façon dont les fonctions et les méthodes sont définies lorsqu'elles doivent être modifiées après leur dénition d'origine.

Nous devons d'abord comprendre qu'en Python, les fonctions sont des objets normaux comme à peu près n'importe quoi d'autre. Cela signifie que vous pouvez les affecter à des variables, les passer par des paramètres ou même leur appliquer d'autres fonctions. Il était typique de vouloir écrire une petite fonction, puis de lui appliquer quelques transformations, générant une nouvelle version modifiée de cette fonction (similaire au fonctionnement de la composition de fonctions en mathématiques).

L'une des motivations originales pour l'introduction des décorateurs était que parce que des fonctions telles que classmethod et staticmethod étaient utilisées pour transformer la définition originale de la méthode, elles nécessitaient une ligne supplémentaire, modifiant la définition originale de la fonction dans une instruction séparée.

De manière plus générale, chaque fois que nous devions appliquer une transformation à une fonction, nous devions l'appeler avec la fonction modifier, puis la réaffecter au même nom avec lequel la fonction avait été définie à l'origine.

Par exemple, si nous avons une fonction appelée original, puis nous avons une fonction qui modifie le comportement de l'original par-dessus, appelée modificateur, nous devons écrire quelque chose comme ce qui suit:

    def original(...):    
    ...
    
    original = modifier(original)


Remarquez comment nous modifions la fonction et la réattribuons au même nom. C'est déroutant, sujet aux erreurs (imaginez que quelqu'un oublie de réaffecter la fonction, ou la réaffecte mais pas dans la ligne immédiatement après la définition de la fonction mais beaucoup plus loin) et encombrant. Pour cette raison, une prise en charge de la syntaxe a été ajoutée au langage.

L'exemple précédent pourrait être réécrit ainsi

    @modifierdef 
    original(...):   
    ...

Cela signifie que les décorateurs ne sont que du sucre syntaxique pour appeler tout ce qui est après le décorateur en tant que premier paramètre du décorateur lui-même, et le résultat serait ce que le décorateur renvoie.


La syntaxe pour les décorateurs améliore considérablement la lisibilité, car maintenant le lecteur de ce code peut trouver toute la dénition de la fonction en un seul endroit. Gardez à l'esprit que la modification manuelle des fonctions comme avant est toujours autorisée.

    En général, essayez d'éviter de réaffecter des valeurs à une fonction déjà
    conçue sans utiliser la syntaxe du décorateur. En particulier, si la
    fonction est réaffectée à autre chose et que cela se produit dans une
    partie éloignée du code (loin de l'endroit où la fonction a été dénie à
    l'origine), cela rendra votre code plus difficile à lire

Conformément à la terminologie Python et à notre exemple, le modificateur est ce que nous appelons le décorateur, et l'original est la fonction décorée, souvent également appelée objet enveloppé.

Alors que la fonctionnalité a été pensée à l'origine pour les méthodes et les fonctions, la syntaxe actuelle permet à tout type d'objet d'être décoré, nous allons donc explorer les décorateurs appliqués aux fonctions, méthodes, générateurs et classes.

Une note finale est que bien que le décorateur de nom soit correct (après tout, le décorateur apporte des modifications, étend ou travaille au-dessus de la fonction enveloppée), il ne doit pas être confondu avec le modèle de conception du décorateur

## Décorateurs de fonction

Les fonctions sont probablement la représentation la plus simple d'un objet Python pouvant être décoré. Nous pouvons utiliser des décorateurs sur les fonctions pour leur appliquer toutes sortes de logiques : nous pouvons valider des paramètres, vérifier des conditions préalables, changer complètement le comportement, modifier sa signature, mettre en cache les résultats (créer une version mémorisée de la fonction d'origine), etc.

À titre d'exemple, nous allons créer un décorateur de base qui implémente un mécanisme de nouvelle tentative, contrôlant une exception particulière au niveau du domaine et réessayant un certain nombre de fois :

In [ ]:
from functools import wraps
from typing import Type

import logging
logger = logging.getLogger(__name__)


class ControlledException(Exception):
    """A generic exception on the program's domain."""


def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised

    return wrapped

L'utilisation de @wraps peut être ignorée pour le moment, car elle sera couverte dans la section Décorateurs efficaces - éviter les erreurs courantes


    L'utilisation de _ dans la boucle for signifie que le numéro est attribué à
    une variable qui ne nous intéresse pas pour le moment, car il n'est pas
    utilisé dans la boucle for (c'est un idiome courant en Python de nommer _
    les valeurs qui sont ignorées)

Le décorateur de nouvelle tentative ne prend aucun paramètre, il peut donc être facilement appliqué à n'importe quelle fonction, comme suit 

In [ ]:
@retry
def run_operation(task):
    """Run a particular task, simulating some failures on its execution."""
    return task.run()

La définition de @retry au-dessus de run_operation est juste du sucre syntaxique que Python fournit pour exécuter run_operation = retry(run_operation)


Dans cet exemple limité, nous pouvons voir comment les décorateurs peuvent être utilisés pour créer une opération de nouvelle tentative générique qui, sous certaines conditions (dans ce cas, représentées comme des exceptions pouvant être liées à des délais d'attente), permettra d'appeler plusieurs fois le code décoré

## Décorateurs de cours

Les classes sont aussi des objets en Python (à vrai dire, à peu près tout est un objet en Python, et il est difficile de trouver un contre-exemple ; cependant, il y a quelques nuances techniques). Cela signifie que les mêmes considérations s'appliquent; ils peuvent aussi être passés par des paramètres, affectés à des variables, demandé certaines méthodes, ou être transformés (décorés).

Les décorateurs de classe ont été introduits dans PEP-3129, et ils ont des considérations très similaires aux décorateurs de fonction que nous venons d'explorer. La seule différence est que lors de l'écriture du code pour ce type de décorateur, nous devons tenir compte du fait que nous recevons une classe en tant que paramètre de la méthode enveloppée, pas une autre fonction.

Nous avons vu comment utiliser un décorateur de classe lorsque nous avons vu le décorateur dataclasses.dataclass,au guide Pythonic Code. Dans ce guide, nous allons apprendre à écrire nos propres décorateurs de classe


Certains praticiens pourraient argumenter que la décoration d'une classe est quelque chose d'assez compliqué et qu'un tel scénario pourrait compromettre la lisibilité car nous déclarerions certains attributs et méthodes dans la classe, mais dans les coulisses, le décorateur pourrait appliquer des changements qui rendraient un complètement différent classer.


Cette évaluation est vraie, mais seulement si cette technique est fortement abusée. Objectivement, ce n'est pas différent des fonctions de décoration ; après tout, les classes ne sont qu'un autre type d'objet dans l'écosystème Python, tout comme les fonctions. Nous passerons en revue les avantages et les inconvénients de ce problème avec les décorateurs dans la section intitulée Décorateurs et séparation des préoccupations, mais pour l'instant, nous explorerons les avantages des décorateurs qui s'appliquent particulièrement aux classes :


* Tous les avantages de la réutilisation du code et du principe DRY. Un cas valide d'un décorateur de classe serait d'imposer que plusieurs classes se conforment à une certaine interface ou à certains critères (en écrivant ces vérifications une seule fois dans un décorateur qui va être appliqué à plusieurs classes).

* Nous pourrions créer des classes plus petites ou plus simples qui sera amélioré plus tard par les décorateurs. 

*  La logique de transformation que nous devons appliquer à une certaine classe sera beaucoup plus facile à maintenir si nous utilisons un décorateur, par opposition à des approches plus compliquées (et souvent à juste titre déconseillées) telles que les méta-classes .

Parmi toutes les applications possibles des décorateurs, nous allons explorer un exemple simple pour montrer le genre de choses pour lesquelles ils peuvent être utiles. Gardez à l'esprit que ce n'est pas le seul type d'application pour les décorateurs de classe, et aussi que le code que je vous montre peut également avoir de nombreuses autres solutions, toutes avec leurs avantages et leurs inconvénients, mais j'ai choisi des décorateurs dans le but d'illustrer leur utilité.

En rappelant nos systèmes d'événements pour la plate-forme de surveillance, nous devons maintenant transformer les données pour chaque événement et les envoyer à un système externe. Cependant, chaque type d'événement peut avoir ses propres particularités lors du choix du mode d'envoi de ses données.

En particulier, l'événement d'une connexion peut contenir des informations sensibles telles que des informations d'identification que nous souhaitons masquer. D'autres champs tels que l'horodatage peuvent également nécessiter des transformations car nous souhaitons les afficher dans un format particulier. Une première tentative pour se conformer à ces exigences serait aussi simple que d'avoir une classe qui correspond à chaque événement particulier et sait comment le sérialiser



In [ ]:
from datetime import datetime
from dataclasses import dataclass


class LoginEventSerializer:
    def __init__(self, event):
        self.event = event

    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": "**redacted**",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M"),
        }


@dataclass
class LoginEvent:
    SERIALIZER = LoginEventSerializer

    username: str
    password: str
    ip: str
    timestamp: datetime

    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()

Ici, nous déclarons une classe qui va être mappée directement avec l'événement de connexion, contenant la logique pour cela - masquer le champ de mot de passe et formater l'horodatage comme requis.

Bien que cela fonctionne et puisse sembler être une bonne option pour commencer, au fur et à mesure que le temps passe et que nous souhaitons étendre notre système, nous trouverons quelques problèmes :

* Trop de classes : à mesure que le nombre d'événements augmente, le nombre de classes de sérialisation augmentera dans le même ordre de grandeur, car elles sont mappées une à une.

* La solution n'est pas assez flexible : si nous devons réutiliser des parties des composants (par exemple, nous devons masquer le mot de passe dans un autre type d'événement qui le possède également), nous devrons l'extraire dans une fonction, mais aussi appelez-le à plusieurs reprises à partir de plusieurs classes, ce qui signifie que nous ne réutilisons pas autant de code après tout.

* Boilerplate : La méthode serialize() devra être présente dans toutes les classes d'événements, appelant le même code. Bien que nous puissions extraire cela dans une autre classe (créer un mixin), cela ne semble pas être une bonne utilisation de l'héritage.


Une solution alternative consiste à construire dynamiquement un objet qui, étant donné un ensemble de filtres (fonctions de transformation) et une instance d'événement, peut le sérialiser en appliquant les filtres à ses champs. Il suffit ensuite de définir les fonctions pour transformer chaque type de champ, et le sérialiseur est créé en composant plusieurs de ces fonctions.

Une fois que nous avons cet objet, nous pouvons décorer la classe afin d'ajouter la méthode serialize(), qui appellera simplement ces objets Serialization avec elle-même :

In [ ]:
from dataclasses import dataclass
from datetime import datetime


def hide_field(field) -> str:
    return "**redacted**"


def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")


def show_original(event_field):
    return event_field


class EventSerializer:
    """Apply the transformations to an Event object based on its properties and
    the definition of the function to apply to each field.
    """

    def __init__(self, serialization_fields: dict) -> None:
        """Created with a mapping of fields to functions.
        Example::
        >>> serialization_fields = {
        ...    "username": str.upper,
        ...    "name": str.title,
        ... }
        Means that then this object is called with::
        >>> from types import SimpleNamespace
        >>> event = SimpleNamespace(username="usr", name="name")
        >>> result = EventSerializer(serialization_fields).serialize(event)
        Will return a dictionary where::
        >>> result == {
        ...     "username": event.username.upper(),
        ...     "name": event.name.title(),
        ... }
        True
        """
        self.serialization_fields = serialization_fields

    def serialize(self, event) -> dict:
        """Get all the attributes from ``event``, apply the transformations to
        each attribute, and place it in a dictionary to be returned.
        """
        return {
            field: transformation(getattr(event, field))
            for field, transformation in self.serialization_fields.items()
        }


class Serialization:
    """A class decorator created with transformation functions to be applied
    over the fields of the class instance.
    """

    def __init__(self, **transformations):
        """The ``transformations`` dictionary contains the definition of how to
        map the attributes of the instance of the class, at serialization time.
        """
        self.serializer = EventSerializer(transformations)

    def __call__(self, event_class):
        """Called when being applied to ``event_class``, will replace the
        ``serialize`` method of this one by a new version that uses the
        serializer instance.
        """

        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)

        event_class.serialize = serialize_method
        return event_class


@Serialization(
    username=str.lower,
    password=hide_field,
    ip=show_original,
    timestamp=format_time,
)
@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime

Remarquez comment le décorateur permet à l'utilisateur de savoir plus facilement comment chaque champ va être traité sans avoir à regarder dans le code d'une autre classe. Juste en lisant les arguments passés au décorateur de classe, nous savons que le nom d'utilisateur et l'adresse IP ne seront pas modifiés, le mot de passe sera masqué et l'horodatage sera formaté


Désormais, le code de la classe n'a pas besoin de la méthode serialize() définie, ni de s'étendre à partir d'un mixin qui l'implémente, puisque le décorateur l'ajoutera. C'est probablement la seule partie qui justifie la création du décorateur de classe car sinon, l'objet Serialization aurait pu être un attribut de classe de LoginEvent, mais le fait qu'il modifie la classe en y ajoutant une nouvelle méthode rend impossible .

## Autres types de décorateur

Maintenant que nous savons ce que signifie la syntaxe @ pour les décorateurs, nous pouvons conclure que ce ne sont pas seulement les fonctions, les méthodes ou les classes qui peuvent être décorées ; en fait, tout ce qui peut être défini, comme les générateurs, les coroutines et même les objets qui ont déjà été décorés, peut être décoré, ce qui signifie que les décorateurs peuvent être empilés.

L'exemple précédent a montré comment les décorateurs peuvent être enchaînés. Nous avons d'abord défini la classe, puis lui avons appliqué @dataclass, qui l'a convertie en une classe de données, agissant comme un conteneur pour ces attributs. Après cela, @Serialization appliquera la logique à cette classe, résultant en une nouvelle classe avec la nouvelle méthode serialize() ajoutée.

Maintenant que nous connaissons les bases des décorateurs, et comment les écrire, nous pouvons passer à des exemples plus complexes. Dans les sections suivantes, nous verrons comment avoir des décorateurs plus flexibles avec des paramètres, et différentes manières de les implémenter

## Décorateurs plus avancés

Avec l'introduction que nous venons d'avoir, nous connaissons maintenant les bases des décorateurs : ce qu'ils sont, et leur syntaxe et sémantique.

Nous nous intéressons maintenant aux utilisations plus avancées des décorateurs qui nous aideront à structurer notre code plus proprement.

Nous verrons que nous pouvons utiliser des décorateurs pour séparer les préoccupations en fonctions plus petites et réutiliser le code, mais pour le faire efficacement, nous aimerions paramétrer les décorateurs (sinon, nous finirions par répéter le code). Pour cela, nous explorerons différentes options sur la façon de transmettre des arguments aux décorateurs.

Après cela, nous pouvons voir quelques exemples de bonnes utilisations des décorateurs.

## Faire passer des arguments aux décorateurs

À ce stade, nous considérons déjà les décorateurs comme un outil puissant en Python. Cependant, ils pourraient être encore plus puissants si nous pouvions simplement leur passer des paramètres afin que leur logique soit encore plus abstraite.

Il existe plusieurs façons d'implémenter des décorateurs qui peuvent prendre des arguments, mais nous allons passer en revue les plus courantes. La première consiste à créer des décorateurs sous forme de fonctions imbriquées avec un nouveau niveau d'indirection, ce qui fait que tout dans le décorateur tombe d'un niveau plus profond. La deuxième approche consiste à utiliser une classe pour le décorateur (c'est-à-dire pour implémenter un objet appelable qui agit toujours comme décorateur).

En général, la seconde approche favorise davantage la lisibilité, car il est plus facile de penser en termes d'objet que trois fonctions imbriquées ou plus travaillant avec des fermetures. Cependant, pour être complet, nous allons explorer les deux, et vous pouvez décider ce qui convient le mieux au problème à résoudre.

## Décorateurs avec fonctions imbriquées

En gros, l'idée générale d'un décorateur est de créer une fonction qui renvoie une autre fonction (en programmation fonctionnelle, les fonctions qui prennent d'autres fonctions comme paramètres sont appelées fonctions d'ordre supérieur, et cela fait référence au même concept dont nous parlons ici ). La fonction interne définie dans le corps du décorateur va être celle qu'on appelle.

Maintenant, si nous souhaitons lui passer des paramètres, nous avons alors besoin d'un autre niveau d'indirection. La première fonction prendra les paramètres, et à l'intérieur de cette fonction, nous en définirons un nouveau, qui sera le décorateur, qui à son tour définira encore une autre nouvelle fonction, à savoir celle qui sera renvoyée en résultat du processus de décoration. Cela signifie que nous aurons au moins trois niveaux de fonctions imbriquées.

Ne vous inquiétez pas si cela ne semble pas clair jusqu'à présent. Après avoir passé en revue les exemples qui sont sur le point de venir, tout deviendra clair

L'un des premiers exemples que nous avons vus de décorateurs implémentait la fonctionnalité de nouvelle tentative sur certaines fonctions. C'est une bonne idée, sauf qu'il y a un problème ; notre implémentation ne nous a pas permis de spécifier le nombre de tentatives, et à la place, c'était un nombre fixe à l'intérieur du décorateur

Maintenant, nous voulons pouvoir indiquer combien de tentatives chaque instance va avoir, et peut-être pourrions-nous même ajouter une valeur par défaut à ce paramètre. Pour ce faire, nous avons besoin d'un autre niveau de fonctions imbriquées — d'abord pour les paramètres, puis pour le décorateur lui-même

C'est parce que nous allons maintenant avoir quelque chose sous la forme suivante

    @retry(arg1, arg2,... )

Et cela doit renvoyer un décorateur car la syntaxe @ appliquera le résultat de ce calcul à l'objet à décorer. Sémantiquement, cela se traduirait par quelque chose comme ceci :

    <original_function> = retry(arg1, arg2, ....)(<original_function>)

Outre le nombre de tentatives souhaitées, nous pouvons également indiquer les types d'exceptions que nous souhaitons contrôler. La nouvelle version du code prenant en charge les nouvelles exigences pourrait ressembler à ceci.



In [ ]:
from functools import wraps
from typing import Sequence, Optional

import logging
logger = logging.getLogger(__name__)


class ControlledException(Exception):
    """A generic exception on the program's domain."""


_DEFAULT_RETRIES_LIMIT = 3


def with_retry(
    retries_limit: int = _DEFAULT_RETRIES_LIMIT,
    allowed_exceptions: Optional[Sequence[Exception]] = None,
):
    allowed_exceptions = allowed_exceptions or (ControlledException,)  # type: ignore

    def retry(operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(retries_limit):
                try:
                    return operation(*args, **kwargs)
                except allowed_exceptions as e:
                    logger.warning(
                        "retrying %s due to %s", operation.__qualname__, e
                    )
                    last_raised = e
            raise last_raised

        return wrapped

    return retry

Voici quelques exemples de la façon dont ce décorateur peut être appliqué aux fonctions, montrant les différentes options qu'il accepte :

In [ ]:
@with_retry()
def run_operation(task):    
  return task.run()
  
@with_retry(retries_limit=5)
def run_with_custom_retries_limit(task):    
  return task.run()


@with_retry(allowed_exceptions=(AttributeError,))
def run_with_custom_exceptions(task):    
  return task.run()
  
@with_retry(    
    retries_limit=4, allowed_exceptions=(ZeroDivisionError,
                                         AttributeError))
def run_with_custom_parameters(task):    
  return task.run()

Utiliser des fonctions imbriquées pour implémenter des décorateurs est probablement la première chose à laquelle nous penserions. Cela fonctionne bien dans la plupart des cas, mais comme vous l'avez peut-être remarqué, l'indentation continue de s'additionner, pour chaque nouvelle fonction que nous créons, donc cela pourrait rapidement conduire à trop de fonctions imbriquées. De plus, les fonctions étant sans état, les décorateurs écrits de cette manière ne contiendront pas nécessairement des données internes, contrairement aux objets.

Il existe une manière différente d'implémenter les décorateurs, qui, au lieu d'utiliser des fonctions imbriquées, utilise des objets, comme nous l'explorons dans la section suivante.

## Objets de décoration

L'exemple précédent requiert trois niveaux de fonctions imbriquées. La première va être une fonction qui reçoit les paramètres du décorateur que nous voulons utiliser. A l'intérieur de cette fonction, le reste des fonctions sont des fermetures qui utilisent ces paramètres, ainsi que la logique du décorateur.


Une implémentation plus propre de ceci serait d'utiliser une classe pour définir le décorateur. Dans ce cas, nous pouvons passer les paramètres dans la méthode __init__, puis implémenter la logique du décorateur sur la méthode magique nommée __call__.

Le code du décorateur ressemblera à celui de l'exemple suivant :

In [ ]:
from functools import wraps
from typing import Optional, Sequence

import logging
logger = logging.getLogger(__name__)


class ControlledException(Exception):
    """A generic exception on the program's domain."""

_DEFAULT_RETRIES_LIMIT = 3


class WithRetry:
    def __init__(
        self,
        retries_limit: int = _DEFAULT_RETRIES_LIMIT,
        allowed_exceptions: Optional[Sequence[Exception]] = None,
    ) -> None:
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException,)

    def __call__(self, operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:
                    logger.warning(
                        "retrying %s due to %s", operation.__qualname__, e
                    )
                    last_raised = e
            raise last_raised

        return wrapped

Et ce décorateur peut être appliqué à peu près comme le précédent, comme ceci :

In [ ]:
@WithRetry(retries_limit=5)
def run_with_custom_retries_limit(task):    
  return task.run()

Il est important de noter comment la syntaxe Python prend effet ici. Tout d'abord, nous créons l'objet, donc avant que l'opération @ ne soit appliquée, l'objet est créé avec ses paramètres qui lui sont transmis. Cela créera un nouvel objet et l'initialisera avec ces paramètres, comme défini dans la méthode init. Après cela, l'opération @ est invoquée, donc cet objet enveloppera la fonction nommée run_with_custom_reries_limit, ce qui signifie qu'il sera passé à la méthode call magic

À l'intérieur de cette méthode d'appel magique, nous avons défini la logique du décorateur comme nous le faisons normalement : nous enveloppons la fonction d'origine, en renvoyant une nouvelle avec la logique que nous voulons à la place

## Décorateurs avec des valeurs par défaut

Dans l'exemple précédent, nous avons vu un décorateur qui prend des paramètres, mais ces arguments ont des valeurs par défaut. La façon dont les décorateurs précédents ont été écrits garantira qu'ils fonctionnent tant que les utilisateurs n'oublient pas les parenthèses pour effectuer l'appel de fonction lors de l'utilisation du décorateur.

Par exemple, si nous voulions uniquement les valeurs par défaut, cela fonctionnerait :

    @retry()
    def my function(): ...

Mais ce ne serait pas


    @retrydef 
    my function(): ...

Vous pourriez vous demander si cela est nécessaire et accepter (peut-être avec une documentation appropriée) que le premier exemple est la façon dont le décorateur est censé être utilisé, et le second est incorrect. Et ce serait bien, mais cela nécessite une attention particulière, sinon des erreurs d'exécution se produiront.

Bien sûr, si le décorateur prend des paramètres qui n'ont pas de valeurs par défaut, alors la deuxième syntaxe n'a pas de sens, et il n'y a qu'une seule possibilité, qui pourrait rendre les choses plus simples.

Alternativement, vous pouvez faire fonctionner le décorateur avec les deux syntaxes. Comme vous l'avez peut-être deviné, cela demande des efforts supplémentaires et, comme toujours, vous devez déterminer si cela en vaut la peine.

Illustrons cela avec un exemple simple qui utilise un décorateur avec des paramètres pour injecter des arguments dans une fonction. Nous définissons une fonction qui prend deux paramètres, et un décorateur qui fait la même chose, et l'idée est d'appeler la fonction sans arguments et de la laisser fonctionner avec les paramètres passés par le décorateur :

    @decorator(x=3, y=4)
    def my_function(x, y):            
        return x + y        
      
    my_function()


Mais bien sûr, nous définissons des valeurs par défaut pour les arguments du décorateur, nous pouvons donc l'appeler sans valeurs. Et nous aimerions aussi l'appeler sans les parenthèses

In [ ]:
from functools import wraps, partial

DEFAULT_X = 1
DEFAULT_Y = 2


def decorator(function=None, *, x=DEFAULT_X, y=DEFAULT_Y):
    if function is None:
        return partial(
            decorator, x=x, y=y
        )  # also lambda f: decorator(f, x=x, y=y)

    @wraps(function)
    def wrapped():
        return function(x, y)

    return wrapped

Notez quelque chose d'important à propos de la signature du décorateur : les paramètres sont des mots-clés uniquement. Cela simplifie beaucoup la dénition du décorateur car nous pouvons supposer que la fonction est None lorsqu'elle est appelée sans arguments (sinon, si nous devions passer les valeurs par position, le premier des paramètres que nous passerions serait confondu avec la fonction). Si nous voulions être plus prudents, au lieu d'utiliser None (ou n'importe quelle valeur sentinelle), nous pourrions inspecter le type de paramètre, affirmer un objet fonction du type attendu, puis déplacer les paramètres en conséquence, mais cela rendrait le décorateur beaucoup plus plus compliqué.

Une autre alternative serait d'abstraire une partie du décorateur enveloppé, puis d'appliquer une application partielle de la fonction (en utilisant functools.partial). Pour mieux expliquer cela, prenons un état intermédiaire et utilisons une fonction lambda qui montre comment les paramètres du décorateur sont appliqués et comment les arguments du décorateur sont "décalés":

In [ ]:
def decorator(function=None, *, x=DEFAULT_X, y=DEFAULT_Y):    
  if function is None:        
    return lambda f: decorator(f, x=x, y=y)     
  
  @wraps(function)    
    def wrapped():        
      return function(x, y)     
  
  return wrapped

Ceci est analogue à l'exemple précédent, dans le sens où nous avons la définition de la fonction encapsulée (comment elle est décorée). Ensuite, si aucune fonction n'est fournie, nous renvoyons une nouvelle fonction qui prend une fonction comme paramètre (f) et renvoie le décorateur avec cette fonction appliquée et le reste des paramètres liés. Ensuite, dans le deuxième appel récursif, la fonction existera et la fonction de décorateur régulière (encapsulée) sera renvoyée à la place

vous pouvez obtenir le même résultat en changeant la définition lambda pour une application partielle de la fonction :

    return partial(decorator, x=x, y=y)


Si cela est trop complexe pour nos cas d'utilisation, nous pouvons toujours décider de faire en sorte que les paramètres de nos décorateurs prennent des valeurs obligatoires.

Dans tous les cas, c'est probablement une bonne idée de dénir les paramètres des décorateurs comme étant uniquement des mots-clés (qu'ils aient des valeurs par défaut ou non). En effet, en général, lors de l'application d'un décorateur, il n'y a pas beaucoup de contexte sur ce que fait chaque valeur, et l'utilisation de valeurs positionnelles peut ne pas donner une expression très significative, il est donc préférable d'être plus expressif et de passer le nom de le paramètre avec la valeur.

De même, si notre décorateur n'a pas l'intention de prendre des paramètres, et que nous voulons être explicites à ce sujet, nous pouvons utiliser la syntaxe que nous avons apprise au guide , Pythonic Code, pour définir la fonction que notre décorateur reçoit comme un simple seul paramètre. Pour notre premier exemple, la syntaxe serait.

    def retry(operation, /): ...

Mais gardez à l'esprit que ce n'est pas strictement recommandé, c'est juste un moyen de vous expliquer comment le décorateur est censé être invoqué.

## Décorateurs de coroutines

Comme expliqué dans l'introduction, puisque pratiquement tout en Python est un objet, alors à peu près tout peut être décoré, et cela inclut également les coroutines.

Cependant, il y a une mise en garde ici, et c'est, comme expliqué dans les guides précédents, la programmation asynchrone en Python introduit quelques différences de syntaxe. Par conséquent, ces différences de syntaxe seront également transmises au décorateur.

En termes simples, si nous devions écrire un décorateur pour une coroutine, nous pourrions simplement nous adapter à la nouvelle syntaxe (n'oubliez pas d'attendre la coroutine encapsulée et de définir l'objet encapsulé comme une coroutine elle-même, ce qui signifie que la fonction interne devra probablement utiliser 'async def' au lieu de simplement 'def').

Le problème est si nous voulons avoir un décorateur largement applicable aux fonctions et aux coroutines. Dans la plupart des cas, créer deux décorateurs serait l'approche la plus simple (et peut-être la meilleure), mais si nous voulions exposer une interface plus simple pour nos utilisateurs (en ayant moins d'objets à retenir), nous pourrions créer un wrapper mince, agissant comme un répartiteur à deux décorateurs internes (non exposés). Ce serait comme créer une façade mais avec un décorateur.

Il n'y a pas de règle générale sur la difficulté de créer un décorateur pour les fonctions et les coroutines, car cela dépend de la logique que nous voulons mettre dans le décorateur lui-même. Par exemple, dans le code ci-dessous, il y a un décorateur qui modifie les paramètres des fonctions qu'il reçoit, et cela fonctionnera aussi bien pour une fonction régulière que pour une coroutine :

In [ ]:
import inspect
import asyncio
from functools import wraps
import time

X, Y = 1, 2


def decorator(callable):
    """Call <callable> with fixed values"""

    @wraps(callable)
    def wrapped():
        return callable(X, Y)

    return wrapped


@decorator
def func(x, y):
    return x + y


@decorator
async def coro(x, y):
    return x + y



Il est cependant important de faire une distinction à propos de la coroutine. Le décorateur recevra la coroutine comme argument appelable, puis l'invoquera avec les paramètres. Cela crée l'objet coroutine (la tâche qui ira à la boucle d'événement), mais il ne l'attend pas, ce qui signifie que celui qui appelle wait coro() finira par attendre la coroutine résultant de ce que le décorateur a enveloppé. Cela signifie que, dans des cas simples comme celui-ci, nous n'avons pas besoin de remplacer la coroutine par une autre coroutine (bien que cela soit généralement recommandé).

Mais encore une fois, cela dépend de ce que nous devons faire. Si nous avons besoin d'une fonction de synchronisation, nous devons attendre que la fonction ou la coroutine se termine pour mesurer le temps, et pour cela nous devrons appeler wait sur elle, ce qui signifie que l'objet wrapper devra à son tour être un coroutine (mais pas le décorateur principal, cependant)

Le code ci-dessous illustre cet exemple à l'aide d'un décorateur qui décide de manière sélective comment envelopper la fonction d'appelant :

In [ ]:
def timing(callable):
    @wraps(callable)
    def wrapped(*args, **kwargs):
        start = time.time()
        result = callable(*args, **kwargs)
        latency = time.time() - start
        return {"latency": latency, "result": result}

    @wraps(callable)
    async def wrapped_coro(*args, **kwargs):
        start = time.time()
        result = await callable(*args, **kwargs)
        latency = time.time() - start
        return {"latency": latency, "result": result}

    if inspect.iscoroutinefunction(callable):
        return wrapped_coro

    return wrapped


@timing
def func2():
    time.sleep(0.1)
    return 42


@timing
async def coro2():
    await asyncio.sleep(0.1)
    return 42

Le deuxième wrapper est requis pour les coroutines. Si nous ne l'avions pas, alors le code aurait deux problèmes. Premièrement, l'appel à callable (sans wait) n'attendrait pas la fin de l'opération, ce qui signifie que les résultats seraient incorrects. Et pire encore, la valeur de la clé de résultat sur le dictionnaire ne serait pas le résultat lui-même, mais la coroutine créée. En conséquence, la réponse serait un dictionnaire, et quiconque essaiera d'appeler cela essaiera d'attendre un dictionnaire, ce qui provoquera une erreur

    En règle générale, il faut remplacer un objet décoré par un autre du même
    genre, c'est-à-dire une fonction par une fonction, et une coroutine par une
    autre coroutine.

Nous devons encore étudier une dernière amélioration qui a été récemment ajoutée à Python, et elle lève certaines des restrictions que sa syntaxe avait

## Syntaxe étendue pour les décorateurs

Python 3.9 a introduit une nouveauté pour les décorateurs, avec PEP-614 (https://www.python.org/dev/peps/pep-0614/), car une grammaire plus générale est autorisée. Avant cette amélioration, la syntaxe pour invoquer les décorateurs (après le @) était limitée à des expressions très limitées, et toutes les expressions Python n'étaient pas autorisées.

Avec ces restrictions levées, nous pouvons maintenant écrire des expressions plus complexes et les utiliser dans nos décorateurs, si nous pensons que cela peut nous faire économiser quelques lignes de code (mais comme toujours, attention à ne pas trop compliquer et à obtenir un format plus compact mais beaucoup plus difficile -ligne à lire).

À titre d'exemple, nous pouvons simplifier certaines des fonctions imbriquées que nous avons généralement pour un simple décorateur qui enregistre un appel de fonction avec ses paramètres. Ici (et à des fins d'illustration uniquement), j'ai remplacé les définitions de fonctions imbriquées, typiques des décorateurs, par deux expressions lambda :


In [ ]:
def _log(f, *args, **kwargs):
    print(f"calling {f.__qualname__!r} with {args=} and {kwargs=}")
    return f(*args, **kwargs)


@(lambda f: lambda *args, **kwargs: _log(f, *args, **kwargs))
def func(x):
    return x + 1

Le document PEP cite quelques exemples pour lesquels cette fonctionnalité peut être utile (comme simplifier les fonctions no-op pour évaluer d'autres expressions, ou éviter l'utilisation de la fonction eval).

La recommandation de ce guide pour cette fonctionnalité est cohérente avec tous les cas dans lesquels une instruction plus compacte peut être obtenue : écrivez la version la plus compacte du code tant que cela ne nuit pas à la lisibilité. Si l'expression du décorateur devient difficile à lire, préférez l'alternative plus verbeuse mais plus simple consistant à écrire deux ou plusieurs fonctions

## Bons usages pour les décorateurs

Dans cette section, nous examinerons quelques modèles courants qui font bon usage des décorateurs. Ce sont des situations courantes pour lesquelles les décorateurs sont un bon choix. Parmi toutes les innombrables applications pour lesquelles les décorateurs peuvent être utilisés, nous en énumérerons quelques-unes, les plus courantes ou les plus pertinentes : 

*  Paramètres de transformation : modifier la signature d'une fonction pour exposer une plus belle API, tout en encapsulant des détails sur la façon dont les paramètres sont traités et transformés en dessous. Il faut être prudent avec cette utilisation des décorateurs, car ce n'est un bon trait que lorsqu'il est intentionnel. Cela signifie que si nous utilisons explicitement des décorateurs pour fournir une bonne signature pour des fonctions qui en avaient une plutôt alambiquée, alors c'est un excellent moyen d'obtenir un code plus propre au moyen de décorateurs. Si, d'un autre côté, la signature d'une fonction a changé par inadvertance à cause d'un décorateur, alors c'est quelque chose que nous voudrions éviter (et nous verrons comment vers la fin du chapitre).

* Traçage du code : enregistrement de l'exécution de une fonction avec ses paramètres. Vous connaissez peut-être plusieurs bibliothèques qui offrent des capacités de traçage et exposent souvent des fonctionnalités telles que des décorateurs à ajouter à nos fonctions. C'est une belle abstraction et une bonne interface à fournir, comme moyen d'intégrer le code avec des parties externes sans trop de perturbations. De plus, c'est une excellente source d'inspiration, nous pouvons donc écrire notre propre fonctionnalité de journalisation ou de traçage en tant que décorateurs. 

* Paramètres de validation : les décorateurs peuvent être utilisés pour valider les types de paramètres (par rapport aux valeurs attendues ou à leurs annotations, par exemple) de manière transparente . Avec l'utilisation de décorateurs, nous pourrions appliquer des conditions préalables pour nos abstractions, en suivant les idées de la conception par contrat.

* Implémentation des opérations de nouvelle tentative : d'une manière similaire à l'exemple que nous avons exploré dans la section précédente.

* Simplifier les classes en déplaçant certaines ( répétitive) en décorateurs : Ceci est lié au principe DRY, que nous reviendrons vers la fin du guide.

Dans les sections suivantes, j'aborderai certains de ces sujets plus en détail

## Adaptation des signatures de fonction

Dans la conception orientée objet, il arrive parfois que des objets avec différentes interfaces doivent interagir. Une solution à ce problème est le modèle de conception d'adaptateur, dont nous discuterons au guide 7, Générateurs, itérateurs et programmation asynchrone, lorsque nous passerons en revue certains des principaux modèles de conception.

Le sujet de cette section est cependant similaire, dans le sens où nous devons parfois adapter non pas des objets, mais des signatures de fonctions.


Imaginez un scénario dans lequel vous travaillez avec du code hérité, et il y a un module qui contient beaucoup de fonctions définies avec une signature complexe (beaucoup de paramètres, passe-partout, etc.). Ce serait bien d'avoir une interface plus propre pour interagir avec ces définitions mais changer beaucoup de fonctions implique un refactor majeur.

Nous pouvons utiliser des décorateurs pour minimiser les différences dans les changements.


Parfois, nous pouvons utiliser des décorateurs comme adaptateur entre notre code et un framework que nous utilisons, si, par exemple, ce framework a les considérations susmentionnées.

imaginez le cas d'un framework qui s'attend à appeler des fonctions définies par nous, en maintenant une certaine interface :

    def resolver_function(root, args, context, info): ...


Maintenant, nous avons cette signature partout et décidons qu'il est préférable pour nous de créer une abstraction à partir de tous ces paramètres qui les encapsule et expose le comportement dont nous avons besoin dans notre application.

Alors maintenant, nous avons beaucoup de fonctions dont la première ligne répète le passe-partout consistant à créer le même objet encore et encore, puis le reste de la fonction n'interagit qu'avec notre objet de domaine:

    def resolver_function(root, args, context, info):    
      helper = DomainObject(root, args, context, info)    
      ...    
      helper.process()


Dans cet exemple, nous pourrions avoir un décorateur changeant la signature de la fonction, afin que nous puissions écrire nos fonctions en supposant que l'objet assistant est passé directement. Dans ce cas, la tâche du décorateur serait d'intercepter les paramètres d'origine, de créer l'objet domaine, puis de passer l'objet assistant à notre fonction. Nous définissons ensuite nos fonctions en supposant que nous ne recevons que l'objet dont nous avons besoin, déjà initialisé.

A savoir, nous aimerions écrire notre code sous cette forme :

    @DomainArgsdef 
    resolver_function(helper):    
      helper.process()   
        ...


Cela fonctionne également dans l'autre sens, par exemple, si le code hérité que nous avons est celui qui prend trop de paramètres et que nous déconstruisons toujours un objet déjà créé, car refactoriser le code hérité serait risqué, alors nous pouvons le faire celle avec un décorateur comme couche intermédiaire.


L'idée est que cette utilisation de décorateurs peut vous aider à écrire des fonctions avec des signatures plus simples et plus compactes.

## Validation des paramètres

Nous avons déjà mentionné que les décorateurs peuvent être utilisés pour valider des paramètres (et même appliquer certaines conditions préalables ou post-conditions sous l'idée de Design by Contract (DbC)), donc à partir de là, vous avez probablement l'idée qu'il est assez courant d'utiliser des décorateurs lors du traitement ou de la manipulation de paramètres.


En particulier, il existe des cas où nous nous retrouvons à créer à plusieurs reprises des objets similaires ou à appliquer des transformations similaires que nous souhaiterions faire abstraction. La plupart du temps, nous pouvons y parvenir en utilisant simplement un décorateur.

## Code de traçage

Lorsque nous parlerons de traçage dans cette section, nous ferons référence à quelque chose de plus général qui concerne l'exécution d'une fonction que nous souhaitons surveiller. Cela peut faire référence à des scénarios dans lesquels nous souhaitons :

*  Tracer l'exécution d'une fonction (par exemple, en enregistrant les lignes qu'elle exécute)

*  Surveiller certaines métriques sur une fonction (telles que l'utilisation du processeur ou l'empreinte mémoire)

* Mesurer le temps d'exécution d'une fonction

* Journal lorsqu'une fonction a été appelée et les paramètres qui lui ont été transmis.

Dans la section suivante, nous explorerons un exemple simple d'un décorateur qui enregistre l'exécution d'une fonction, y compris son nom et le temps qu'il a fallu pour s'exécuter.

## Décorateurs efficaces – éviter les erreurs courantes

Bien que les décorateurs soient une fonctionnalité intéressante de Python, ils ne sont pas exempts de problèmes s'ils sont mal utilisés. Dans cette section, nous verrons quelques problèmes courants à éviter afin de créer des décorateurs efficaces.

## Préservation des données sur l'objet enveloppé d'origine

L'un des problèmes les plus courants lors de l'application d'un décorateur à une fonction est que certaines des propriétés ou attributs de la fonction d'origine ne sont pas conservés, ce qui entraîne des effets secondaires indésirables et difficiles à suivre.

Pour illustrer cela, nous montrons un décorateur qui est en charge de la journalisation lorsque la fonction est sur le point de s'exécuter :

In [2]:
!pip install log

ERROR: Could not find a version that satisfies the requirement log (from versions: none)
ERROR: No matching distribution found for log


In [3]:
import logging
logger = logging.getLogger(__name__)


def trace_decorator(function):
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)

    return wrapped

Maintenant, imaginons que nous ayons une fonction avec ce décorateur qui lui est appliqué. On pourrait d'abord penser que rien de cette fonction n'est modifié par rapport à sa dénition d'origine :

    @trace_decoratordef 
    process_account(account_id: str):    
        """Process an account by Id."""    
        logger.info("processing account %s", account_id)    
        ...


Mais peut-être qu'il y a des changements.


Le décorateur n'est pas censé modifier quoi que ce soit par rapport à la fonction d'origine, mais il s'avère que, puisqu'il contient une faille, il modifie en fait son nom et sa docstring, entre autres propriétés.


Essayons d'obtenir de l'aide pour cette fonction :

In [4]:
@trace_decorator
def process_account(account_id: str):
    """Process an account by Id."""
    logger.info("processing account %s", account_id)
    ...

In [5]:
help(process_account)

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)



Et regardons comment ça s'appelle :

In [6]:
process_account.__qualname__

'trace_decorator.<locals>.wrapped'

Et aussi, les annotations pour la fonction d'origine ont été perdues :

In [7]:
process_account.__annotations__

{}

Nous pouvons voir que, puisque le décorateur change en fait la fonction d'origine pour une nouvelle (appelée encapsulée), ce que nous voyons en fait, ce sont les propriétés de cette fonction au lieu de celles de la fonction d'origine.

If we apply a decorator like this one to multiple functions, all with different names, they will all end up being called wrapped, which is a major concern (for example, if we want to log or trace the function, this will make debugging even harder).

Si nous appliquons un décorateur comme celui-ci à plusieurs fonctions, toutes avec des noms différents, elles finiront toutes par être appelées encapsulées, ce qui est une préoccupation majeure (par exemple, si nous voulons journaliser ou tracer la fonction, cela rendra le débogage même Plus fort).



Un autre problème est que si nous placions des docstrings avec des tests sur ces fonctions, elles seraient remplacées par celles du décorateur. En conséquence, les docstrings avec le test que nous voulons ne s'exécuteront pas lorsque nous appelons notre code avec le module doctest (comme nous l'avons vu au guide 1, Introduction, Formatage du code et Outils).

La solution est simple, cependant. Il suffit d'appliquer le décorateur wraps dans la fonction interne (wrapped), en lui disant qu'il s'agit en fait d'une fonction wrapping :

In [8]:
from functools import wraps

def trace_decorator(function):
    """Log when a function is being called."""

    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)

    return wrapped


@trace_decorator
def process_account(account_id: str):
    """Process an account by Id."""
    logger.info("processing account %s", account_id)
    ...


Maintenant, si nous vérifions les propriétés, nous obtiendrons ce que nous attendions en premier lieu. Consultez l'aide pour la fonction, comme ceci

In [9]:
help(process_account)

Help on function process_account in module __main__:

process_account(account_id: str)
    Process an account by Id.



In [10]:
process_account.__qualname__

'process_account'

Surtout, nous avons récupéré les tests unitaires que nous aurions pu avoir sur les docstrings ! En utilisant le décorateur wraps, nous pouvons également accéder à la fonction originale non modifiée sous l'attribut __wrapped__. Bien qu'il ne doive pas être utilisé en production, il peut être utile dans certains tests unitaires lorsque nous voulons vérifier la version non modifiée de la fonction.

En général, pour les décorateurs simples, la façon dont nous utiliserions functools.wraps suivrait généralement la formule ou la structure générale suivante :

In [11]:
def decorator(original_function):
    @wraps(original_function)
    def decorated_function(*args, **kwargs):
        # modifications done by the decorator ...
        return original_function(*args, **kwargs)

    return decorated_function

## Gérer les effets secondaires chez les décorateurs

Dans cette section, nous apprendrons qu'il est conseillé d'éviter les effets secondaires dans le corps du décorateur. Il y a des cas où cela pourrait être acceptable, mais l'essentiel est qu'en cas de doute, décidez de ne pas le faire, pour les raisons qui sont expliquées plus loin. Tout ce que le décorateur doit faire en dehors de la fonction qu'il décore doit être placé dans la définition de fonction la plus interne, sinon il y aura des problèmes lors de l'importation. Néanmoins, ces effets secondaires sont parfois nécessaires (ou même souhaités) pour s'exécuter au moment de l'importation, et l'avers s'applique.


Nous verrons des exemples des deux, et où chacun s'applique. En cas de doute, faites preuve de prudence et retardez tous les effets secondaires jusqu'au tout dernier moment, juste après l'appel de la fonction encapsulée.

Ensuite, nous verrons quand ce n'est pas une bonne idée de placer une logique supplémentaire en dehors de la fonction encapsulée.

## Mauvaise gestion des effets secondaires dans un décorateur

Imaginons le cas d'un décorateur qui a été créé dans le but de se connecter lorsqu'une fonction a commencé à s'exécuter, puis de consigner son temps d'exécution :

In [ ]:
def traced_function_wrong(function):
    """An example of a badly defined decorator."""
    logger.info("started execution of %s", function)
    start_time = time.time()

    @wraps(function)
    def wrapped(*args, **kwargs):
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs", function, time.time() - start_time
        )
        return result

    return wrapped


Maintenant, nous allons appliquer le décorateur à une fonction régulière, en pensant que cela fonctionnera juste :

In [ ]:
import time

@traced_function_wrong
def process_with_delay(callback, delay=0):
    logger.info("sleep(%d)", delay)
    return callback

Ce décorateur contient un bogue subtil mais critique. Tout d'abord, importons la fonction, appelons-la plusieurs fois et voyons ce qui se passe :


    >>> from decorator_side_effects_1 import process_with_delay
    INFO:started execution of <function process_with_delay at 0x...>

Juste en important la fonction, nous remarquerons que quelque chose ne va pas. La ligne de journalisation ne devrait pas être là, car la fonction n'a pas été invoquée.

Maintenant, que se passe-t-il si nous exécutons la fonction et voyons combien de temps cela prend pour s'exécuter ? En fait, nous nous attendrions à ce que le fait d'appeler plusieurs fois la même fonction donne des résultats similaires :


    >>> main()
    ...
    INFO:function <function process_with_delay at 0x> took 8.67s

    >>> main()
    ...
    INFO:function <function process_with_delay at 0x> took 13.39s
    
    >>> main()
    ...
    INFO:function <function process_with_delay at 0x> took 17.01s


Chaque fois que nous exécutons la même fonction, cela prend de plus en plus de temps ! À ce stade, vous avez probablement déjà remarqué l'erreur (maintenant évidente)

Rappelez-vous la syntaxe pour les décorateurs. @traced_function_wrong signifie en fait ce qui suit :

    process_with_delay = traced_function_wrong(process_with_delay)

Et cela fonctionnera lorsque le module sera importé. Par conséquent, l'heure qui est définie dans la fonction sera l'heure à laquelle le module a été importé. Les appels successifs calculeront la différence de temps entre l'heure d'exécution et l'heure de début d'origine. Il se connectera également au mauvais moment, et non au moment où la fonction est réellement appelée.


Heureusement, le correctif est également très simple : il suffit de déplacer le code à l'intérieur de la fonction encapsulée afin de retarder son exécution :

In [ ]:
def traced_function(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs", function, time.time() - start_time
        )
        return result

    return wrapped


@traced_function
def call_with_delay(callback, delay=0):
    logger.info("sleep(%d)", delay)
    return callback

Avec cette nouvelle version, les problèmes précédents sont résolus.


Si les actions du décorateur avaient été différentes, les résultats auraient pu être bien plus désastreux. Par exemple, si cela nécessite que vous enregistriez les événements et que vous les envoyiez à un service externe, cela échouera certainement à moins que la configuration n'ait été exécutée juste avant son importation, ce que nous ne pouvons garantir. Même si nous le pouvions, ce serait une mauvaise pratique. La même chose s'applique si le décorateur a tout autre type d'effet secondaire, comme la lecture d'un fichier, l'analyse d'une configuration, et bien d'autres.

## Exiger des décorateurs avec des effets secondaires

Parfois, des effets secondaires sur les décorateurs sont nécessaires, et nous ne devrions pas retarder leur exécution jusqu'à la dernière minute possible, car cela fait partie du mécanisme qui leur est nécessaire pour fonctionner.

Un scénario courant lorsque nous ne voulons pas retarder l'effet secondaire des décorateurs est lorsque nous devons enregistrer des objets dans un registre public qui sera disponible dans le module.

Par exemple, en revenant à notre exemple de système d'événements précédent, nous voulons maintenant rendre uniquement certains événements disponibles dans le module, mais pas tous. Dans la hiérarchie des événements, nous pourrions vouloir avoir des classes intermédiaires qui ne sont pas des événements réels que nous voulons traiter sur le système, mais à la place certaines de leurs classes dérivées.


Au lieu de marquer chaque classe selon qu'elle sera traitée ou non, nous pourrions explicitement enregistrer chaque classe via un décorateur. Dans ce cas, nous avons une classe pour tous les événements liés aux activités d'un utilisateur. Cependant, il ne s'agit que d'une table intermédiaire pour les types d'événements que nous voulons réellement, à savoir UserLoginEvent et UserLogoutEvent :

In [ ]:
EVENTS_REGISTRY = {}


def register_event(event_cls):
    """Place the class for the event into the registry to make it accessible in
    the module.
    """
    EVENTS_REGISTRY[event_cls.__name__] = event_cls
    return event_cls


class Event:
    """A base event object"""


class UserEvent:
    TYPE = "user"


@register_event
class UserLoginEvent(UserEvent):
    """Represents the event of a user when it has just accessed the system."""


@register_event
class UserLogoutEvent(UserEvent):
    """Event triggered right after a user abandoned the system."""

Lorsque nous regardons le code précédent, il semble que EVENTS_REGISTRY soit vide, mais après avoir importé quelque chose de ce module, il sera rempli avec toutes les classes qui se trouvent sous le décorateur register_event :

    >>> from decorator_side_effects_2 import EVENTS_REGISTRY
    
    >>> EVENTS_REGISTRY
    
    {'UserLoginEvent': decorator_side_effects_2.UserLoginEvent, 'UserLogoutEvent': decorator_side_effects_2.UserLogoutEvent}


Cela peut sembler difficile à lire, voire trompeur, car EVENTS_REGISTRY aura sa valeur finale au moment de l'exécution, juste après l'importation du module, et nous ne pouvons pas facilement prédire sa valeur en regardant simplement le code.


Bien que cela soit vrai, dans certains cas, ce modèle est justifié. En fait, de nombreux frameworks Web ou bibliothèques bien connues l'utilisent pour travailler et exposer des objets ou les rendre disponibles. Cela dit, soyez conscient de ce risque, si vous mettez en œuvre quelque chose de similaire dans vos propres projets : la plupart du temps, une solution alternative serait préférable. 

Il est également vrai que dans ce cas, le décorateur ne change pas l'objet emballé. ou modifier la façon dont il fonctionne de quelque façon que ce soit. Cependant, la note importante ici est que si nous devions faire quelques modifications et définir une fonction interne qui modifie l'objet encapsulé, nous voudrions probablement toujours le code qui enregistre l'objet résultant en dehors de celui-ci. Remarquez l'utilisation du mot dehors. Cela ne veut pas nécessairement dire avant, cela ne fait tout simplement pas partie de la même fermeture; mais c'est dans la portée extérieure, donc ce n'est pas retardé jusqu'à l'exécution

## Créer des décorateurs qui fonctionneront toujours

Il existe plusieurs scénarios différents auxquels les décorateurs peuvent s'appliquer. Il peut également arriver que nous ayons besoin d'utiliser le même décorateur pour des objets qui entrent dans ces différents scénarios multiples, par exemple, si nous voulons réutiliser notre décorateur et l'appliquer à une fonction, une classe, une méthode ou un static méthode.

Si nous créons le décorateur, en pensant simplement à ne prendre en charge que le premier type d'objet que nous voulons décorer, nous pourrions remarquer que le même décorateur ne fonctionne pas aussi bien sur un type d'objet différent. Un exemple typique est celui où nous créons un décorateur à utiliser sur une fonction, puis nous voulons l'appliquer à une méthode d'une classe, seulement pour réaliser que cela ne fonctionne pas. Un scénario similaire peut se produire si nous concevons notre décorateur pour une méthode, et que nous souhaitons ensuite qu'il s'applique également aux méthodes statiques ou aux méthodes de classe.


Lors de la conception des décorateurs, nous pensons généralement à la réutilisation du code, nous voudrons donc également utiliser ce décorateur pour les fonctions et les méthodes.Définir nos décorateurs avec la signature *args et **kwargs les fera fonctionner dans tous les cas car c'est le type de signature le plus générique que nous puissions avoir. Cependant, nous pouvons parfois vouloir ne pas l'utiliser et définir à la place la fonction d'emballage du décorateur en fonction de la signature de la fonction d'origine, principalement pour deux raisons : 

*  Elle sera plus lisible car elle ressemble à la fonction d'origine.

*  Il doit en fait faire quelque chose avec les arguments, donc recevoir *args et **kwargs ne serait pas pratique.

Considérons le cas dans lequel nous avons de nombreuses fonctions dans notre base de code qui nécessitent la création d'un objet particulier à partir d'un paramètre. Par exemple, nous passons une chaîne et initialisons un objet pilote avec elle, à plusieurs reprises. Ensuite on pense pouvoir supprimer la duplication en utilisant un décorateur qui se chargera de convertir ce paramètre en conséquence.

Dans l'exemple suivant, nous prétendons que DBDriver est un objet qui sait comment se connecter et exécuter des opérations sur une base de données, mais il a besoin d'une chaîne de connexion. Les méthodes que nous avons dans notre code sont conçues pour recevoir une chaîne avec les informations de la base de données et nous obligent à toujours créer une instance de DBDriver. L'idée du décorateur est qu'il va prendre la place de cette conversion automatiquement - la fonction continuera à recevoir une chaîne, mais le décorateur créera un DBDriver et le transmettra à la fonction, donc en interne nous pouvons supposer que nous recevons le objet dont nous avons besoin directement.

Un exemple d'utilisation de ceci dans une fonction est montré dans la liste suivante :

In [12]:
from functools import wraps

import logging
logger = logging.getLogger(__name__)


class DBDriver:
    def __init__(self, dbstring: str) -> None:
        self.dbstring = dbstring

    def execute(self, query: str) -> str:
        return f"query {query} at {self.dbstring}"


def inject_db_driver(function):
    """This decorator converts the parameter by creating a ``DBDriver``
    instance from the database dsn string.
    """

    @wraps(function)
    def wrapped(dbstring):
        return function(DBDriver(dbstring))

    return wrapped


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function")


Il est facile de vérifier que si nous passons une chaîne à la fonction, nous obtenons le résultat par une instance de DBDriver, donc le décorateur fonctionne comme prévu :

In [13]:
run_query("test_OK")

'query test_function at test_OK'

Mais maintenant, nous voulons réutiliser ce même décorateur dans une méthode de classe, où nous retrouvons le même problème :

In [16]:
class DataHandler:
    """The decorator will not work for methods as it is defined."""

    @inject_db_driver
    def run_query(self, driver):
        return driver.execute(self.__class__.__name__)




Nous essayons d'utiliser ce décorateur, seulement pour nous rendre compte qu'il ne fonctionne pas :

In [17]:
DataHandler().run_query("test_fails")

TypeError: ignored

Quel est le problème ? 

La méthode de la classe est définie avec un argument supplémentaire : self

Les méthodes ne sont qu'un type particulier de fonction qui reçoit self (l'objet sur lequel elles sont définies) comme premier paramètre.

Par conséquent, dans ce cas, le décorateur (conçu pour fonctionner avec un seul paramètre, nommé dbstring) interprétera que self, et appellera la méthode en passant la chaîne à la place de self, et rien à la place du deuxième paramètre , à savoir la chaîne que nous passons.

Pour résoudre ce problème, nous devons créer un décorateur qui fonctionnera également pour les méthodes et les fonctions, et nous le faisons en le définissant comme un objet décorateur qui implémente également le descripteur de protocole.

Les descripteurs sont expliqués en détail au guide 7, Générateurs, itérateurs et programmation asynchrone, donc pour l'instant, nous pouvons simplement considérer cela comme une recette qui fera fonctionner le décorateur.


La solution est d'implémenter le décorateur comme un objet de classe et de faire de cet objet une description, en implémentant la méthode __get__ :

In [ ]:

from functools import wraps
from types import MethodType


class DBDriver:
    def __init__(self, dbstring: str) -> None:
        self.dbstring = dbstring

    def execute(self, query: str) -> str:
        return f"query {query} at {self.dbstring}"


class inject_db_driver:
    """Convert a string to a DBDriver instance and pass this to the wrapped
    function.
    """

    def __init__(self, function) -> None:
        self.function = function
        wraps(self.function)(self)

    def __call__(self, dbstring):
        return self.function(DBDriver(dbstring))

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.__class__(MethodType(self.function, instance))


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function_2")


class DataHandler:
    @inject_db_driver
    def run_query(self, driver):
        return driver.execute("test_method_2")

Les détails sur les descripteurs seront expliqués au guide 6, Tirer le meilleur parti de nos objets avec des descripteurs, mais pour les besoins de cet exemple, nous pouvons maintenant dire que ce qu'il fait est en fait de relier l'appelable qu'il décore à une méthode, ce qui signifie qu'il lier la fonction à l'objet, puis recréer le décorateur avec ce nouvel appelable.

Pour les fonctions, cela fonctionne toujours, car il n'appellera pas du tout la méthode __get__.

## Décorateurs et code propre

Maintenant que nous en savons plus sur les décorateurs, comment les écrire et éviter les problèmes courants, il est temps de les faire passer au niveau supérieur et de voir comment nous pouvons tirer parti de ce que nous avons appris pour obtenir un meilleur logiciel.

Nous avons brièvement abordé ce sujet dans les sections précédentes, mais il s'agissait d'exemples plus proches du code, car les suggestions faisaient référence à la façon de rendre des lignes (ou sections) spécifiques du code plus lisibles.

Les sujets abordés à partir de maintenant portent sur des principes de conception plus généraux. Nous avons déjà visité certaines de ces idées dans les guides précédents, mais la perspective ici est de comprendre comment nous utilisons les décorateurs à de telles fins.

## Composition sur héritage

Nous avons déjà discuté brièvement du fait qu'en général, il est préférable d'avoir la composition plutôt que l'héritage car ce dernier pose certains problèmes de couplage des composants du code. Dans le livre Design Patterns: Elements of Reusable Object-Oriented Software (DESIG01) , la plupart des idées autour du modèle de conception sont basées sur l'idée suivante : 

  
* privilégier la composition à l'héritage de classe.

Dans le guide 2 , Pythonic Code, j'ai introduit l'idée d'utiliser la méthode magique __getattr__ pour résoudre dynamiquement les attributs sur les objets. J'ai également donné l'exemple selon lequel cela pourrait être utilisé pour résoudre automatiquement les attributs en fonction d'une convention de nommage si cela était requis par un framework externe, par exemple. Explorons deux versions différentes de la résolution de ce problème.


Pour cet exemple, supposons que nous interagissions avec un framework qui a la convention de nommage d'appeler des attributs avec le préfixe "resolve_" pour résoudre un attribut, mais nos objets de domaine n'ont que ces attributs sans le préfixe "resolve_" . 

Clairement, nous ne voulons pas écrire beaucoup de méthodes répétitives nommées "resolve_x" pour chaque attribut que nous avons, donc la première idée est de tirer parti de la méthode magique __getattr__ susmentionnée et de la placer dans une classe de base :

In [18]:
from dataclasses import dataclass


class BaseResolverMixin:
    def __getattr__(self, attr: str):
        if attr.startswith("resolve_"):
            *_, actual_attr = attr.partition("resolve_")
        else:
            actual_attr = attr
        try:
            return self.__dict__[actual_attr]
        except KeyError as e:
            raise AttributeError from e
  
@dataclass
class Customer(BaseResolverMixin):
    customer_id: str
    name: str
    address: str

Cela fera l'affaire, mais pouvons-nous faire mieux ? Nous pourrions concevoir un décorateur de classe pour définir cette méthode directement :

In [29]:
from dataclasses import dataclass


def _resolver_method(self, attr):
    """The resolution method of attributes that will replace __getattr__."""
    if attr.startswith("resolve_"):
        *_, actual_attr = attr.partition("resolve_")
    else:
        actual_attr = attr
    try:
        return self.__dict__[actual_attr]
    except KeyError as e:
        raise AttributeError from e


def with_resolver(cls):
    """Set the custom resolver method to a class."""
    cls.__getattr__ = _resolver_method
    return cls


@dataclass
@with_resolver
class Customer:
    customer_id: str
    name: str
    address: str

Les deux versions respecteraient le comportement suivant :

In [30]:
customer = Customer("1", "name", "address")
print(customer.resolve_customer_id)
print(customer.resolve_name)

1
name


Premièrement, nous avons la méthode de résolution en tant que fonction autonome qui respecte la signature de l'apparence du __getattr__ d'origine (c'est pourquoi j'ai même conservé self comme nom du premier paramètre, pour que cette fonction devienne intentionnellement une méthode).

Le reste du code semble assez simple. Notre décorateur ne définit la méthode que sur la classe qu'il reçoit par paramètre, puis nous appliquons le décorateur à notre classe sans avoir à utiliser l'héritage.

En quoi est-ce un peu mieux que l'exemple précédent ? Pour commencer, nous pouvons affirmer que l'utilisation du décorateur implique que nous utilisons la composition (prendre une classe, la modifier et en renvoyer une nouvelle) par rapport à l'héritage, de sorte que notre code est moins couplé à la classe de base que nous avions au début.

De plus, on peut dire que l'utilisation de l'héritage (via une classe mixin) dans le premier exemple était plutôt ctitieuse. Nous n'utilisions pas l'héritage pour créer une version plus spécialisée de la classe, mais uniquement pour tirer parti de la méthode __getattr__. Ce serait mauvais pour deux raisons (complémentaires) : d'abord, l'héritage n'est pas le meilleur moyen de réutiliser du code. Un bon code est réutilisé en ayant de petites abstractions cohérentes, sans créer de hiérarchies.

Deuxièmement, rappelez-vous des guides précédents que la création d'une sous-classe devrait suivre l'idée de spécialisation, le type de relation « est un ». Réfléchissez si, d'un point de vue conceptuel, un client est bien un BaseResolverMivin (et qu'est-ce que c'est, d'ailleurs ?). Pour éclaircir ce deuxième point, imaginez que vous ayez une hiérarchie comme celle-ci :


    class Connection: pass
    class EncryptedConnection(Connection): pass

Dans ce cas, l'utilisation de l'héritage est sans doute correcte, après tout, une connexion cryptée est un type de connexion plus spécifique. Mais quel serait un type plus spécifique de BaseResolverMixin ? Il s'agit d'une classe mixin, il est donc prévu qu'elle soit mélangée dans la hiérarchie avec d'autres classes (en utilisant l'héritage multiple). L'utilisation de cette classe de mix-in est purement pragmatique et à des fins d'implémentation. Ne vous méprenez pas, il s'agit d'un livre pragmatique, vous devrez donc faire face à des classes de mix-in dans votre expérience professionnelle, et il est parfaitement normal de les utiliser, mais si nous pouvons éviter cette abstraction purement implémentationnelle, et remplacez-le par quelque chose qui ne compromet pas nos objets de domaine (dans ce cas la classe Customer), c'est encore mieux.

Il y a une autre capacité passionnante du nouveau design, qui est l'extensibilité. Nous avons vu comment les décorateurs peuvent être paramétrés. Imaginez la exibilité que nous pourrions atteindre dans notre conception si nous permettions au décorateur de définir n'importe quelle fonction de résolveur, pas seulement celle que nous avons définie.

## Décorateurs et séparation des préoccupations

Le dernier point de la liste précédente est si important qu'il mérite une section à part. Nous avons déjà exploré l'idée de réutiliser le code et remarqué qu'un élément clé de la réutilisation du code est d'avoir des composants cohérents. Cela signifie qu'ils devraient avoir le niveau minimum de responsabilité : faire une chose, une seule chose, et la faire bien. Plus nos composants sont petits, plus ils sont réutilisables et plus ils peuvent être appliqués dans un contexte différent sans entraîner de comportement supplémentaire qui provoquera des couplages et des dépendances, ce qui rendra le logiciel rigide.


Pour vous montrer ce que cela signifie, reprenons l'un des décorateurs que nous avons utilisés dans un exemple précédent. Nous avons créé un décorateur qui a tracé l'exécution de certaines fonctions avec un code similaire au suivant :



In [31]:
import functools
import time

import logging
logger = logging.getLogger(__name__)


def traced_function(function):
    @functools.wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs",
            function.__qualname__,
            time.time() - start_time,
        )
        return result

    return 

Or, ce décorateur, pendant qu'il travaille, a un problème : il fait plus d'une chose. Il enregistre qu'une fonction particulière vient d'être invoquée, et enregistre également le temps qu'il a fallu pour s'exécuter. Chaque fois que nous utilisons ce décorateur, nous assumons ces deux responsabilités, même si nous n'en voulions qu'une. Celle-ci doit être décomposée en décorateurs plus petits, chacun avec une responsabilité plus spécifique et limitée :

In [ ]:
import time
from functools import wraps

import logging
logger = logging.getLogger(__name__)


def log_execution(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        return function(*kwargs, **kwargs)

    return wrapped


def measure_time(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        start_time = time.time()
        result = function(*args, **kwargs)

        logger.info(
            "function %s took %.2f",
            function.__qualname__,
            time.time() - start_time,
        )
        return result

    return wrapped


@measure_time
@log_execution
def operation():
    time.sleep(3)
    logger.info("running operation...")
    return 33

Remarquez comment l'ordre dans lequel les décorateurs sont appliqués est également important.

.